# Preprocess data for Alpha/Beta diversity calculation

In [1]:
import numpy as np 
import pandas as pd 

In [2]:
## load species abundance

df_species=pd.read_csv('./data/metaphlan2_species_level_abundance_classified.csv',index_col=0)

## Construct the phylogenetic tree from for species in the metaphlan2 result

This will enable calculation of Alpha/Beta diversity using phylogenetic metric

The strategey is first to map species names to assemblyIDs in NCBI, then map the assemblyID to taxonID. 
Finally we can create a phylogenetic tree using the taxonID and python package ete3


### Map species names to assemblyIDs

In [3]:
# load file that map species name to assemblyID

df_TaxAssem=pd.read_csv('/media/nucleoid/raw_data/xin/tools/Strainphlan/utils/species2genomes.txt',sep='\t1\t',header=None)

#create a dictionary that map metaphlan2 tax names to assmbly IDs
dict_taxToassem=dict(zip(df_TaxAssem.index,df_TaxAssem[0]))
dict_taxToassem

## map the species in dic-species to assmebly ID 
assemblyList=[]

for i in df_species.index: 
    spec=i.split('|')[-1]
    if spec in dict_taxToassem.keys(): 
        assemblyList.append(dict_taxToassem[spec])
    else: 
        for s in dict_taxToassem.keys(): 
            if spec in s: 
                assemblyList.append(s.split('\t')[-1])
                break
        
len(assemblyList)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


440

### Create phylogenetic tree of taxonID using ete3

In [4]:
# reload the curated conversion dataframe
df_curated=pd.read_csv('./data/speciesToTaxonID_sandborn.csv',index_col=0)

In [5]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()

#get rid of two strains to avoid confusion 
taxonID=list(df_curated.taxonID)

#create and save the phylogenetic tree in nw format
tree = ncbi.get_topology(taxonID)

print tree.get_ascii(attributes=['taxid'])
#tree.write(format=1, outfile="./data/phylotree_sandborn.nw")


                                                /-1161422
                                               |
                                          /671232-862966
                                         |     |
                                         |      \-1095730
                                         |
                                         |--1105032
                                         |
                                         |--888833
                                         |
                                         |--1116231
                                         |
                                         |--760858
                                         |
                                         |--889201
                                         |
                                         |--889204
                                         |
                                         |--889206
                                         |
                         

In [6]:
#change the species into taxonID for metaplan output so this can be later used fro diversity calculation
df_species.index=list(df_curated.taxonID)
print df_species.shape
#remove two taxonID that are not on the tip of phylogenetic tree
df_species=df_species.drop(187327)
df_species=df_species.drop(818)

print df_species.shape

#df_species.to_csv('../data/metaphlan2_species_abundance_taxonID.csv')

(440, 301)
(438, 301)
